In [1]:
!git clone https://github.com/CompVis/attribute-control.git
%cd attribute-control/notebooks
!pip install -q -r ../requirements.txt

Cloning into 'attribute-control'...
remote: Enumerating objects: 177, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 177 (delta 23), reused 43 (delta 14), pack-reused 108 (from 1)
Receiving objects: 100% (177/177), 114.55 MiB | 20.70 MiB/s, done.
Resolving deltas: 100% (24/24), done.
/Users/vamshi.bonagiri/Desktop/stuff/overspecified/attribute-control/notebooks


/Users/vamshi.bonagiri/Desktop/stuff/.venv/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [4]:
import sys

import torch
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output

sys.path.append('..')
from attribute_control import EmbeddingDelta
from attribute_control.model import SDXL
from attribute_control.prompt_utils import get_mask, get_mask_regex

torch.set_float32_matmul_precision('high')

DEVICE = 'mps'
DTYPE = torch.float16

In [5]:
model = SDXL(
    pipeline_type='diffusers.StableDiffusionXLPipeline',
    model_name='stabilityai/stable-diffusion-xl-base-1.0',
    pipe_kwargs={ 'torch_dtype': DTYPE, 'variant': 'fp16', 'use_safetensors': True },
    device=DEVICE
)

Loading pipeline components...: 100%|██████████| 7/7 [00:00<00:00, 14.38it/s]


In [6]:
delta = EmbeddingDelta(model.dims)
state_dict = torch.load('../pretrained_deltas/person_age.pt')
delta.load_state_dict(state_dict['delta'])
delta = delta.to(DEVICE)

In [ ]:
# prompt = 'a photo of a beautiful man'
prompt = 'a man driving a bike'
# The delta is applied to this regex pattern in the positive prompt
# If you don't feel comfortable with regex, use get_mask(prompt, target) instead
pattern_target = r'\b(man)\b'
prompt_negative = None # Optional negative prompt
seed = 42
scales = np.linspace(-2, 2, num=5) # [0.0, 1.0]

# Delta application delay
# Set to 0 to apply the delta for the whole sampling process
# Set to something between 0 and 1 to skip applying the delta for the first steps (e.g., first 20% of steps for 0.2)
# If you prefer a minor change to the overall image (e. g., just the face changing when modifying age), set to ~0.2
# If you'd rather want major changes that capture all correlations such as the background changing with age, set to 0.0
delay_relative = 0.20

# Sample from the set of provided scales
characterwise_mask = get_mask_regex(prompt, pattern_target)
emb = model.embed_prompt(prompt)
emb_neg = None if prompt_negative is None else model.embed_prompt(prompt_negative)
imgs = []
for alpha in scales:
    img = model.sample_delayed(
        # Multiple deltas can simply be applied by stacking delta.apply() calls with different deltas
        embs=[delta.apply(emb, characterwise_mask, alpha)],
        embs_unmodified=[emb],
        embs_neg=[emb_neg],
        delay_relative=delay_relative,
        generator=torch.manual_seed(seed),
        guidance_scale=7.5
    )[0]
    imgs.append(img)

    # Display outputs
    clear_output()
    plt.figure(figsize=(max(10, 4 * len(imgs)), 5))
    for i, (alpha, img) in enumerate(zip(scales, imgs)):
        plt.subplot(1, len(imgs), i + 1)
        plt.imshow(img)
        plt.title(f'scale = {alpha:.2f}' if alpha != 0 else 'default generation (scale = 0)', fontsize=10)
        plt.axis('off')
    plt.tight_layout(pad=0.5, h_pad=1.0, w_pad=0.5)
    plt.subplots_adjust(top=0.9)
    plt.show()

 20%|██        | 2/10 [00:03<00:14,  1.76s/it]